In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

# Récupération des données

In [ ]:
# Source : https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/index.html
#  

# Récupère la liste des départements
url = 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/index.html'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Récupère les liens des départements (dans un select id="selectDep")
departements = soup.find('select', {'id': 'selectDep'}).find_all('option')
departements = [departement['value'] for departement in departements if departement['value'] != '']

print(departements)
# exemple : ['84/01/index.html',...]

['84/01/index.html', '32/02/index.html', '84/03/index.html', '93/04/index.html', '93/05/index.html', '93/06/index.html', '84/07/index.html', '44/08/index.html', '76/09/index.html', '44/10/index.html', '76/11/index.html', '76/12/index.html', '93/13/index.html', '28/14/index.html', '84/15/index.html', '75/16/index.html', '75/17/index.html', '24/18/index.html', '75/19/index.html', '27/21/index.html', '53/22/index.html', '75/23/index.html', '75/24/index.html', '27/25/index.html', '84/26/index.html', '28/27/index.html', '24/28/index.html', '53/29/index.html', '94/2A/index.html', '94/2B/index.html', '76/30/index.html', '76/31/index.html', '76/32/index.html', '75/33/index.html', '76/34/index.html', '53/35/index.html', '24/36/index.html', '24/37/index.html', '84/38/index.html', '27/39/index.html', '75/40/index.html', '24/41/index.html', '84/42/index.html', '84/43/index.html', '52/44/index.html', '24/45/index.html', '76/46/index.html', '75/47/index.html', '76/48/index.html', '52/49/index.html',

In [232]:
# Pour chaque département, récupère les liens des circonscriptions (id="selectCir")
circonscriptions = []
for departement in departements:
    url = f'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/{departement}'
    response = requests.get(url)
    if response.status_code != 200:
        print(f'Erreur {response.status_code} pour {url}')
        continue
    soup = BeautifulSoup(response.text, 'html.parser')
    options = soup.find('select', {'id': 'selectCir'}).find_all('option')
    # ici, on récupère par exemple ["0101/index.html",...]
    dpt = departement.replace('/index.html', '')
    circonscriptions += [f'{dpt}/{option["value"]}' for option in options if option['value'] != '']

print(circonscriptions)
# exemple : ['84/01/0101/index.html',...]

['84/01/0101/index.html', '84/01/0102/index.html', '84/01/0103/index.html', '84/01/0104/index.html', '84/01/0105/index.html', '32/02/0201/index.html', '32/02/0202/index.html', '32/02/0203/index.html', '32/02/0204/index.html', '32/02/0205/index.html', '84/03/0301/index.html', '84/03/0302/index.html', '84/03/0303/index.html', '93/04/0401/index.html', '93/04/0402/index.html', '93/05/0501/index.html', '93/05/0502/index.html', '93/06/0601/index.html', '93/06/0602/index.html', '93/06/0603/index.html', '93/06/0604/index.html', '93/06/0605/index.html', '93/06/0606/index.html', '93/06/0607/index.html', '93/06/0608/index.html', '93/06/0609/index.html', '84/07/0701/index.html', '84/07/0702/index.html', '84/07/0703/index.html', '44/08/0801/index.html', '44/08/0802/index.html', '44/08/0803/index.html', '76/09/0901/index.html', '76/09/0902/index.html', '44/10/1001/index.html', '44/10/1002/index.html', '44/10/1003/index.html', '76/11/1101/index.html', '76/11/1102/index.html', '76/11/1103/index.html',

In [233]:
# Liste des URLs des pages pour chaque circonscription
urls = [f'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/{circo}' for circo in circonscriptions]
print(urls)

['https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/84/01/0101/index.html', 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/84/01/0102/index.html', 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/84/01/0103/index.html', 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/84/01/0104/index.html', 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/84/01/0105/index.html', 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/32/02/0201/index.html', 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/32/02/0202/index.html', 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographique/32/02/0203/index.html', 'https://www.resultats-elections.interieur.gouv.fr/legislatives2024/ensemble_geographiq

In [234]:
# Récupère les données d'une page
def get_data(url):
    response = requests.get(url)
    response.encoding = 'utf-8'
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table')
    thead = table.find('thead')
    th = thead.find_all('th')
    columns = [th_.text for th_ in th]

    tbody = table.find('tbody')
    rows = tbody.find_all('tr')
    data = []
    for row in rows:
        cells = row.find_all('td')
        data.append([cell.text.strip() for cell in cells])

    # A partir du fil d'Ariane de la page, récupère le nom du département et la circonscription
    fil_ariane = soup.find('nav', {'class': 'fr-breadcrumb fr-mb-4v'})
    ol = fil_ariane.find('ol')
    li = ol.find_all('li')
    departement = li[2].text.strip()
    circonscription = li[3].text.strip()

    return pd.DataFrame(data, columns=columns), departement, circonscription

In [235]:
# test : récupère les données de la 1ere page
data, departement, circonscription = get_data(urls[0])
print(f'{departement}, {circonscription}')
print(data)

Ain (01), 1ère circonscription (0101)
    Liste des candidats Nuance    Voix % Inscrits % Exprimés Elu(e)
0      M. Xavier BRETON     LR  33 889      39,02      56,48    OUI
1  M. Christophe MAÎTRE     RN  26 116      30,07      43,52    NON


In [236]:
# Récupère les résultats de toutes les pages
dfs = []
for url in urls:
    data, departement, circonscription = get_data(url)
    data['departement'] = departement
    data['circonscription'] = circonscription
    dfs.append(data)
    print(f'{len(dfs)}/{len(urls)}', end='\r')


# Statistiques

In [246]:
# Concatène les résultats
resultats = pd.concat(dfs)

# Convertit les colonnes en numérique
resultats['Voix'] = resultats['Voix'].str.replace('\u202f', '').str.replace(' ', '').astype(int)
resultats['% Inscrits'] = resultats['% Inscrits'].str.replace(',', '.').str.replace('%', '').astype(float)
resultats['% Exprimés'] = resultats['% Exprimés'].str.replace(',', '.').str.replace('%', '').astype(float)

# Renomme "Liste des candidats" en "Candidat"
resultats = resultats.rename(columns={'Liste des candidats': 'Candidat'})

In [238]:
# Candidats élus avec les plus gros scores relatifs (en % Exprimés)
resultats[resultats['Elu(e)'] == 'OUI'].sort_values(by='% Exprimés', ascending=False).head(5)

,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e),departement,circonscription
0,M. Davy RIMANE,REG,8422,16.95,100.00,OUI,Guyane (973),2ème circonscription (97302)
0,M. Jean-Philippe NILOR,REG,21620,25.61,86.58,OUI,Martinique (972),4ème circonscription (97204)
0,M. Jiovanny WILLIAM,REG,18552,23.09,81.97,OUI,Martinique (972),1ère circonscription (97201)
0,Mme Estelle YOUSSOUFFA,DVD,13640,29.87,79.48,OUI,Mayotte (976),1ère circonscription (97601)
0,M. Olivier SERVA,DVG,18043,23.81,77.59,OUI,Guadeloupe (971),1ère circonscription (97101)


In [247]:
# Candidats élus avec les plus petits scores relatifs (en % Exprimés)
resultats[resultats['Elu(e)'] == 'OUI'].sort_values(by='% Exprimés', ascending=True).head(5)

,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e),departement,circonscription
0,M. Fabrice BRUN,DVD,20414,25.20,34.65,OUI,Ardèche (07),3ème circonscription (0703)
0,M. René PILATO,UG,20288,24.08,35.75,OUI,Charente (16),1ère circonscription (1601)
0,Mme Zahia HAMDANE,UG,18538,23.97,35.76,OUI,Somme (80),2ème circonscription (8002)
0,M. Laurent CROIZIER,ENS,19324,25.49,36.18,OUI,Doubs (25),1ère circonscription (2501)
0,M. Peio DUFAU,UG,27117,25.43,36.28,OUI,Pyrénées-Atlantiques (64),6ème circonscription (6406)


In [248]:
# Affiche les candidats élus avec le plus petit % d'inscrits
resultats[resultats['Elu(e)'] == 'OUI'].sort_values(by='% Inscrits', ascending=True).head(5)

,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e),departement,circonscription
0,Mme Eléonore CAROIT,ENS,9947,12.16,53.27,OUI,Français établis hors de France (ZZ),2ème circonscription (ZZ02)
0,Mme Caroline YADAN,ENS,18302,12.29,52.70,OUI,Français établis hors de France (ZZ),8ème circonscription (ZZ08)
0,Mme Amélia LAKRAFI,ENS,17055,14.98,53.23,OUI,Français établis hors de France (ZZ),10ème circonscription (ZZ10)
0,M. Frantz GUMBS,DVC,3942,15.62,56.02,OUI,Saint-Martin/Saint-Barthélemy (ZX),1ère circonscription (ZX01)
0,M. Davy RIMANE,REG,8422,16.95,100.00,OUI,Guyane (973),2ème circonscription (97302)


In [249]:
# affiche les candidats de Ardèche (07) / 3ème circonscription (0703)
resultats[(resultats['departement'] == 'Ardèche (07)') & (resultats['circonscription'] == '3ème circonscription (0703)')]

,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e),departement,circonscription
0,M. Fabrice BRUN,DVD,20414,25.20,34.65,OUI,Ardèche (07),3ème circonscription (0703)
1,M. Cyrille GRANGIER,RN,20379,25.16,34.59,NON,Ardèche (07),3ème circonscription (0703)
2,Mme Florence PALLOT,UG,18125,22.37,30.76,NON,Ardèche (07),3ème circonscription (0703)


In [242]:
# Recherche les candidats ayant obtenu la plus petite avance absolue (en voix) et relative (en % exprimés)

# on s'assure que le tableau est trié par circonscription, puis par voix
resultats_sorted = resultats.sort_values(by=['departement', 'circonscription', 'Voix'], ascending=[True, True, False])

# affiche les premières lignes pour vérifier
resultats_sorted.head()

,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e),departement,circonscription
0,M. Xavier BRETON,LR,33889,39.02,56.48,OUI,Ain (01),1ère circonscription (0101)
1,M. Christophe MAÎTRE,RN,26116,30.07,43.52,NON,Ain (01),1ère circonscription (0101)
0,M. Romain DAUBIÉ,ENS,38973,38.24,55.09,OUI,Ain (01),2ème circonscription (0102)
1,M. Andréa KOTARAC,RN,31766,31.17,44.91,NON,Ain (01),2ème circonscription (0102)
0,Mme Olga GIVERNET,ENS,32958,39.16,63.00,OUI,Ain (01),3ème circonscription (0103)


In [243]:
# pour comparer facilement les candidats au sein d'une même circonscription, on va créer un dictionnaire avec pour clé le couple (département, circonscription) et pour valeur un tableau de candidats (dataframe)
dict_candidats = {}
for index, row in resultats_sorted.iterrows():
    key = (row['departement'], row['circonscription'])
    if key not in dict_candidats:
        dict_candidats[key] = []
    dict_candidats[key].append(row)

In [244]:
# pour chaque couple (département, circonscription), on va chercher les candidats ayant obtenu la plus petite avance
for key, candidats in dict_candidats.items():
    if len(candidats) < 2:
        continue
    # Calucle les avances absolue et relative
    dict_candidats[key][0]['Avance absolue (Voix)'] = candidats[0]['Voix']-candidats[1]['Voix']
    dict_candidats[key][0]['Avance relative (% Exprimés)'] = candidats[0]['% Exprimés']-candidats[1]['% Exprimés']

# Crée un dataframe avec les résultats
resultats_avances = pd.concat([pd.DataFrame(candidats) for candidats in dict_candidats.values()])

# Affiche les candidats ayant obtenu la plus petite avance absolue
resultats_avances.sort_values(by='Avance absolue (Voix)').head()

,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e),departement,circonscription,Avance absolue (Voix),Avance relative (% Exprimés)
0,M. Fabrice BRUN,DVD,20414,25.20,34.65,OUI,Ardèche (07),3ème circonscription (0703),35.0,0.06
0,M. René LIORET,RN,28677,33.51,50.04,OUI,Côte-d'Or (21),5ème circonscription (2105),44.0,0.08
0,M. Fabrice BARUSSEAU,UG,26441,31.58,50.06,OUI,Charente-Maritime (17),3ème circonscription (1703),63.0,0.12
0,M. Aurélien DUTREMBLE,RN,26523,32.79,50.06,OUI,Saône-et-Loire (71),3ème circonscription (7103),65.0,0.12
0,Mme Florence JOUBERT,RN,22589,33.74,50.08,OUI,Dordogne (24),3ème circonscription (2403),75.0,0.16


In [ ]:
# Candidats ayant les plus petites avances absolues
for index, row in resultats_avances.sort_values(by='Avance absolue (Voix)').head(3).iterrows():
    print(f'{row["departement"]}, {row["circonscription"]}')
    print(f'Avance absolue : {int(row["Avance absolue (Voix)"])} voix')
    print(f'Avance relative : {row["Avance relative (% Exprimés)"]:.2f} % des voix exprimées')
    display(resultats[(resultats['departement'] == row['departement']) & (resultats['circonscription'] == row['circonscription'])].drop(columns=['departement', 'circonscription']))

Ardèche (07), 3ème circonscription (0703)
Avance absolue : 35 voix
Avance relative : 0.06 % des voix exprimées


,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e)
0,M. Fabrice BRUN,DVD,20414,25.20,34.65,OUI
1,M. Cyrille GRANGIER,RN,20379,25.16,34.59,NON
2,Mme Florence PALLOT,UG,18125,22.37,30.76,NON


Côte-d'Or (21), 5ème circonscription (2105)
Avance absolue : 44 voix
Avance relative : 0.08 % des voix exprimées


,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e)
0,M. René LIORET,RN,28677,33.51,50.04,OUI
1,M. Didier PARIS,ENS,28633,33.46,49.96,NON


Charente-Maritime (17), 3ème circonscription (1703)
Avance absolue : 63 voix
Avance relative : 0.12 % des voix exprimées


,Candidat,Nuance,Voix,% Inscrits,% Exprimés,Elu(e)
0,M. Fabrice BARUSSEAU,UG,26441,31.58,50.06,OUI
1,M. Stéphane MORIN,RN,26378,31.50,49.94,NON
